In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab02.ipynb")

# Lab 2: SQL

In this lab, we are going to practice viewing, sorting, grouping, and merging tables with SQL. We will explore the money donated during the 2016 election using the [Federal Election Commission's public records](https://www.fec.gov/data/). You will be connecting to a sqlite database containing the data. The data we will be working with in this lab is quite small (16MB); however, it's taken from a larger database that is more than few GBs.


### Due Date

This assignment is due at **Friday, February 12th** at 11:59 pm.

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about this assignment, we ask that you **write your solutions individually**. If you discuss the assignment with others, please **include their names** in the cell below.

**Collaborators:** *list names here*

In [2]:
import pandas as pd
import sqlalchemy
from ds100_utils import fetch_and_cache
from pathlib import Path

# Part I – SQL

### Connecting to SQLite

Download the database to local disk drive; this will take about a minute.

In [3]:
db_url = 'https://s3.amazonaws.com/berkeley-ds100/fec_nyc.sqlite'
db_filename = 'fec_nyc.db'
data_dir = Path('data')
fetch_and_cache(db_url, db_filename, data_dir=data_dir)
db_path = data_dir / db_filename

Downloading... Done!


The following cell will connect to the above database using the SQL Alchemy library. This library is more general than the `sqlite3` module from the standard library because it can connect to several different database management systems, such as MySQL and PostgreSQL. It also supports an advanced feature for generating queries called an [object relational mapper](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html) or ORM, which we won't discuss in this course but is quite useful for application development.

In [4]:
engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()

To check that everything is working we will list the available tables.

In [5]:
engine.table_names()

['cand', 'comm', 'indiv_sample_nyc']

## Table Descriptions

Here is a list of the tables in the database.  Each table links to the documentation on the [FEC page](https://www.fec.gov/data/browse-data/) for the dataset.

If you'd like, you can consult the FEC page for the descriptions of the tables to find out what the correspondence is.

- [`cand`](https://www.fec.gov/campaign-finance-data/candidate-summary-file-description/): Candidates table. Contains names and party affiliation.
- [`comm`](https://www.fec.gov/campaign-finance-data/committee-summary-file-description/): Committees table. Contains committee names and types.
- [`indiv_sample_nyc`](https://www.fec.gov/campaign-finance-data/contributions-individuals-file-description/): All individual contributions from New York City.

We can see the schemas of these tables with the following code.

In [6]:
for exp in connection.execute('SELECT sql FROM sqlite_master;'):
    print(exp[0])

CREATE TABLE "cand" (
    cand_id character varying(9),
    cand_name text,
    cand_pty_affiliation character varying(3),
    cand_election_yr integer,
    cand_office_st character varying(2),
    cand_office character(1),
    cand_office_district integer,
    cand_ici character(1),
    cand_status character(1),
    cand_pcc character varying(9),
    cand_st1 text,
    cand_st2 text,
    cand_city text,
    cand_st character varying(2),
    cand_zip character varying(10)
)
CREATE TABLE "comm"(
  "cmte_id" TEXT,
  "cmte_nm" TEXT,
  "tres_nm" TEXT,
  "cmte_st1" TEXT,
  "cmte_st2" TEXT,
  "cmte_city" TEXT,
  "cmte_st" TEXT,
  "cmte_zip" TEXT,
  "cmte_dsgn" TEXT,
  "cmte_tp" TEXT,
  "cmte_pty_affiliation" TEXT,
  "cmte_filing_freq" TEXT,
  "org_tp" TEXT,
  "connected_org_nm" TEXT,
  "cand_id" TEXT
)
CREATE TABLE indiv_sample_nyc (
    cmte_id character varying(9),
    amndt_ind character(1),
    rpt_tp character varying(3),
    transaction_pgi character(5),
    image_num bigint,
    trans

We can also look at the tables themselves.

The individual donations made by NY residents are shown below.

In [7]:
# We use `LIMIT 5` to avoid loading a huge table.
# Although our tables shouldn't get too large to display,
# this is generally good practice when working in the
# notebook environment.  Jupyter notebooks don't handle
# very large outputs well. 
query = """
SELECT * from indiv_sample_nyc LIMIT 5
"""
pd.read_sql(query, engine)

,cmte_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,name,city,state,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub_id
0,C00445015,N,Q1,P,15951128130,15,IND,"SINGER, TRIPP MR.",NEW YORK,NY,...,"ATLANTIC MAILBOXES, INC.",OWNER,01302015,1000,,A-CF13736,1002485,,,4041420151241812398
1,C00510461,N,Q1,P,15951129284,15E,IND,"SIMON, DANIEL A",NEW YORK,NY,...,N/A,RETIRED,03292015,400,C00401224,VN8JBDDJBA8,1002590,,* EARMARKED CONTRIBUTION: SEE BELOW,4041420151241813640
2,C00422410,N,Q1,P,15970352211,15,IND,"ABDUL RAUF, FEISAL",NEW YORK,NY,...,THE CORDOBA INITIATIVE,CHAIRMAN,03042015,250,,VN8A3DBSYG6,1003643,,,4041620151241914560
3,C00510461,N,Q1,P,15951129280,15,IND,"SCHWARZER, FRANK",NEW YORK,NY,...,METRO HYDRAULIC JACK CO,SALES,01162015,100,,VN8JBDAP4C4,1002590,,* EARMARKED CONTRIBUTION: SEE BELOW,4041420151241813630
4,C00510461,N,Q1,P,15951129281,15,IND,"SCHWARZER, FRANK",NEW YORK,NY,...,METRO HYDRAULIC JACK CO,SALES,02162015,100,,VN8JBDBRDG3,1002590,,* EARMARKED CONTRIBUTION: SEE BELOW,4041420151241813632


Similarly, the database of candidates is given below.

In [18]:
query = '''
SELECT *
FROM cand
LIMIT 5
'''
pd.read_sql(query,engine)

,cand_id,cand_name,cand_pty_affiliation,cand_election_yr,cand_office_st,cand_office,cand_office_district,cand_ici,cand_status,cand_pcc,cand_st1,cand_st2,cand_city,cand_st,cand_zip
0,H0AK00097,"COX, JOHN R.",REP,2014,AK,H,0,C,N,C00525261,P.O. BOX 1092,,ANCHOR POINT,AK,99556
1,H0AL02087,"ROBY, MARTHA",REP,2016,AL,H,2,I,C,C00462143,PO BOX 195,,MONTGOMERY,AL,36101
2,H0AL02095,"JOHN, ROBERT E JR",IND,2016,AL,H,2,C,N,,1465 W OVERBROOK RD,,MILLBROOK,AL,36054
3,H0AL05049,"CRAMER, ROBERT E ""BUD"" JR",DEM,2008,AL,H,5,,P,C00239038,PO BOX 2621,,HUNTSVILLE,AL,35804
4,H0AL05163,"BROOKS, MO",REP,2016,AL,H,5,I,C,C00464149,7610 FOXFIRE DRIVE,,HUNTSVILLE,AL,35802


And lastly, the table of committees is given below.

In [19]:
query = '''
SELECT *
FROM comm
LIMIT 5
'''
pd.read_sql(query,engine)

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id
0,C00000059,HALLMARK CARDS PAC,ERIN BROWER,2501 MCGEE,MD#288,KANSAS CITY,MO,64108,U,Q,UNK,M,C,,
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,20001,B,Q,,M,M,AMERICAN MEDICAL ASSOCIATION,
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,TOM RITTER,3528 W RENO,,OKLAHOMA CITY,OK,73107,U,N,,Q,L,TEAMSTERS LOCAL UNION 886,
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,"C. RICHARD BONEBRAKE, M.D.",623 SW 10TH AVE,,TOPEKA,KS,66612,U,Q,UNK,Q,T,,
4,C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...,"VIDYA KORA, M.D.","322 CANAL WALK, CANAL LEVEL",,INDIANAPOLIS,IN,46202,U,Q,,Q,M,,


In CS61A or CS88, we saw basic queries similar to what we see below, which gives only the id and name of candidates from the Democratic party.

In [20]:
query = '''
SELECT cand_id, cand_name
FROM cand
WHERE cand_pty_affiliation = 'DEM'
LIMIT 5
'''
pd.read_sql(query,engine)

,cand_id,cand_name
0,H0AL05049,"CRAMER, ROBERT E ""BUD"" JR"
1,H0AL07086,"SEWELL, TERRYCINA ANDREA"
2,H0AL07094,"HILLIARD, EARL FREDERICK JR"
3,H0AR01091,"GREGORY, JAMES CHRISTOPHER"
4,H0AR01109,"CAUSEY, CHAD"


---
## Question 1

For this question we will use SQL to extract data out of the `indiv_sample_nyc` table. The schema for this table is given below.

In [21]:
for exp in connection.execute('SELECT sql FROM sqlite_master WHERE name="indiv_sample_nyc";'):
    print(exp[0])

CREATE TABLE indiv_sample_nyc (
    cmte_id character varying(9),
    amndt_ind character(1),
    rpt_tp character varying(3),
    transaction_pgi character(5),
    image_num bigint,
    transaction_tp character varying(3),
    entity_tp character varying(3),
    name text,
    city text,
    state character(2),
    zip_code character varying(12),
    employer text,
    occupation text,
    transaction_dt character varying(9),
    transaction_amt integer,
    other_id text,
    tran_id text,
    file_num bigint,
    memo_cd text,
    memo_text text,
    sub_id bigint
)


### Question 1a

Let's start by looking at 2016 election contributions made by Donald Trump, who was a NY resident during that year. Write a SQL statement that will return the cmte_id, transaction_amt, and name for every contribution made by any donor with "DONALD" and "TRUMP" in their name.


Hint: You'll need to use `WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%'`.

<!--
BEGIN QUESTION
name: q1a
-->

In [22]:
query_q1a='''
SELECT
    ... 
    ... 
    ...
FROM indiv_sample_nyc
WHERE ...
'''

# BEGIN SOLUTION NO PROMPT
query_q1a='''
SELECT 
    cmte_id,
    transaction_amt,
    name
FROM indiv_sample_nyc
WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%'
'''
# END SOLUTION

res_q1a = pd.read_sql(query_q1a, engine)
res_q1a

,cmte_id,transaction_amt,name
0,C00230482,2600,"DONALD, TRUMP"
1,C00230482,2600,"DONALD, TRUMP"
2,C00014498,9000,"TRUMP, DONALD"
3,C00494229,2000,"TRUMP, DONALD MR"
4,C00571869,2700,"TRUMP, DONALD J."
...,...,...,...
152,C00608489,5,DONALD J TRUMP FOR PRESIDENT INC
153,C00608489,5,DONALD J TRUMP FOR PRESIDENT INC
154,C00608489,5,DONALD J TRUMP FOR PRESIDENT INC
155,C00608489,5,DONALD J TRUMP FOR PRESIDENT INC


In [ ]:
grader.check("q1a")

### Question 1b

If we look at the list above, it appears that some donations were not by Donald Trump himself, but instead by an entity called "DONALD J TRUMP FOR PRESIDENT INC". If we look at the list of donors, we see that our query only seems to have picked up one such anomalous name.

In [25]:
res_q1a['name'].value_counts()

TRUMP, DONALD J.                    133
DONALD J TRUMP FOR PRESIDENT INC     15
TRUMP, DONALD                         4
DONALD, TRUMP                         2
TRUMP, DONALD J MR                    1
TRUMP, DONALD J MR.                   1
TRUMP, DONALD MR                      1
Name: name, dtype: int64

Revise your query from above so that the 15 donations made by "DONALD J TRUMP FOR PRESIDENT INC" do not appear. Your resulting table should have 142 rows. 

Hint: Consider using `NOT LIKE`

<!--
BEGIN QUESTION
name: q1b
-->

In [26]:
query_q1b = '''
SELECT
    ... 
    ... 
    ...
FROM indiv_sample_nyc
WHERE ...
'''
# BEGIN SOLUTION NO PROMPT
query_q1b = '''
SELECT 
    cmte_id,
    transaction_amt,
    name
FROM indiv_sample_nyc
WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%' AND name NOT LIKE '%INC%'
'''
# END SOLUTION

res_q1b = pd.read_sql(query_q1b, engine)
res_q1b

,cmte_id,transaction_amt,name
0,C00230482,2600,"DONALD, TRUMP"
1,C00230482,2600,"DONALD, TRUMP"
2,C00014498,9000,"TRUMP, DONALD"
3,C00494229,2000,"TRUMP, DONALD MR"
4,C00571869,2700,"TRUMP, DONALD J."
...,...,...,...
137,C00580100,9752,"TRUMP, DONALD J."
138,C00580100,2574,"TRUMP, DONALD J."
139,C00580100,23775,"TRUMP, DONALD J."
140,C00580100,2000000,"TRUMP, DONALD J."


In [ ]:
grader.check("q1b")

### Question 1c

As a review of some of the key ideas from CS61A/CS88, let's refine our query above.

We see that many of the contributions above were to the same committee. Create a new SQL query that returns the total amount that Donald Trump contributed to each committee. Your table should have three columns `cmte_id`, `total_amount`, and `num_donations`, where `total_amount` is the total amount contributed to that committee, and `num_donations` is the total number of donations. Your table should be sorted in decreased order of `total_amount`.

Hint: Use `GROUP BY` as covered in CS61A and CS88.

Hint: Use `COUNT(*)` as covered in CS61A and CS88.

Hint: Use `SUM()` as covered in CS61A and CS88.

Hint: Use `ORDER BY` as covered in CS61A and CS88.

Hint: Use `DESC` as covered in CS61A and CS88.

Hint: Your `WHERE` clause should be the same as question 1b.

<!--
BEGIN QUESTION
name: q1c
-->

In [43]:
query_q1c = '''
SELECT
    ... 
    ... 
    ...
FROM indiv_sample_nyc
WHERE ...
...
...
'''

# BEGIN SOLUTION NO PROMPT
query_q1c = '''
SELECT 
    cmte_id,
    SUM(transaction_amt) as total_amount,
    COUNT(*) as num_donations
FROM indiv_sample_nyc
WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%' AND name NOT LIKE '%INC%'
GROUP BY cmte_id
ORDER BY total_amount DESC
'''
# END SOLUTION

res_q1c = pd.read_sql(query_q1c, engine)
res_q1c

,cmte_id,total_amount,num_donations
0,C00580100,18633157,131
1,C00055582,10000,1
2,C00014498,9000,1
3,C00571869,5400,2
4,C00230482,5200,2
5,C00136457,5000,1
6,C00034033,5000,1
7,C00554949,2600,1
8,C00494229,2000,1
9,C00369033,1000,1


In [ ]:
grader.check("q1c")

### Question 1d

Now we'd like to know the name of each committee, i.e. we want to know the name of the identity that Donald Trump was making contributions to. Modify your query from question 1c so that it also shows the name of each committee. That is, your table should have four columns `cmte_id`, `total_amount`, `num_donations`, and `cmte_nm`.

The names of the committees are given in the `cmte_nm` column of the `comm` table.

Hint: You'll need to join the `indiv_sample_nyc` and `comm` tables.

Hint: One approach is a nested select statement (a.k.a. subquery) as discussed in lecture.

<!--
BEGIN QUESTION
name: q1d
-->

In [33]:
query_q1d = '''
...
'''

# BEGIN SOLUTION NO PROMPT
# Solution 1
query_q1d = '''
SELECT 
    cmte_id,
    SUM(transaction_amt) as total_amount,
    COUNT(*) as num_donations,
    cmte_nm
FROM 
        (SELECT *
        FROM indiv_sample_nyc, comm
        WHERE indiv_sample_nyc.cmte_id == comm.cmte_id)
WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%' AND name NOT LIKE '%INC%'
GROUP BY cmte_id
ORDER BY total_amount DESC
'''

# Solution 2
query_q1d = '''
SELECT 
    indiv_sample_nyc.cmte_id,
    SUM(transaction_amt) as total_amount,
    COUNT(*) as num_donations,
    cmte_nm
FROM 
    indiv_sample_nyc JOIN comm ON indiv_sample_nyc.cmte_id = comm.cmte_id
WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%' AND name NOT LIKE '%INC%'
GROUP BY indiv_sample_nyc.cmte_id
ORDER BY total_amount DESC
'''
# END SOLUTION

res_q1d = pd.read_sql(query_q1d, engine)
res_q1d

,cmte_id,total_amount,num_donations,cmte_nm
0,C00580100,18633157,131,"DONALD J. TRUMP FOR PRESIDENT, INC."
1,C00055582,10000,1,NY REPUBLICAN FEDERAL CAMPAIGN COMMITTEE
2,C00014498,9000,1,REPUBLICAN PARTY OF IOWA
3,C00571869,5400,2,DONOVAN FOR CONGRESS
4,C00230482,5200,2,GRASSLEY COMMITTEE INC
5,C00136457,5000,1,NEW HAMPSHIRE REPUBLICAN STATE COMMITTEE
6,C00034033,5000,1,SOUTH CAROLINA REPUBLICAN PARTY
7,C00554949,2600,1,FRIENDS OF DAVE BRAT INC.
8,C00494229,2000,1,HELLER FOR SENATE
9,C00369033,1000,1,TEXANS FOR SENATOR JOHN CORNYN INC


In [ ]:
grader.check("q1d")

## Question 2

### Question 2a

Let's select transactions in the `indiv_sample_nyc` table with a `cmte_id` (committee id) that ends in 5. Then, groupby the transactions in the `indiv_sample_nyc` table with the `cmte_id` column, and count how many transactions and how much total amount of contribution there are for each `cmte_id`. Select the top 5 committees ordered by the transaction count.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>committee_id</th>
      <th>total_amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C00575795</td>
      <td>25099091</td>
      <td>110657</td>
    </tr>
    <tr>
      <th>1</th>
      <td>C00000935</td>
      <td>6989835</td>
      <td>36237</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C00035675</td>
      <td>316019</td>
      <td>3417</td>
    </tr>
    <tr>
      <th>3</th>
      <td>C00040535</td>
      <td>492567</td>
      <td>3277</td>
    </tr>
    <tr>
      <th>4</th>
      <td>C00012245</td>
      <td>499659</td>
      <td>2128</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2a
-->

In [44]:
query_q2a='''
SELECT
    ... AS committee_id,
    ... AS total_amount,
    ... AS count
FROM indiv_sample_nyc
...
...
...
...
'''

# BEGIN SOLUTION NO PROMPT
query_q2a='''
SELECT
    cmte_id AS committee_id,
    sum(transaction_amt) AS total_amount,
    count(*) AS count
FROM indiv_sample_nyc
WHERE cmte_id LIKE '%5'
GROUP BY cmte_id
ORDER BY count DESC
LIMIT 5
'''
# END SOLUTION

res_q2a = pd.read_sql(query_q2a, engine)
res_q2a

,committee_id,total_amount,count
0,C00575795,25099091,110657
1,C00000935,6989835,36237
2,C00035675,316019,3417
3,C00040535,492567,3277
4,C00012245,499659,2128


In [ ]:
grader.check("q2a")

### Question 2b

When you printed the schema for the tables, you may have noticed that both the `cand` and `comm` tables have a `cand_id` column. Let's try joining these two tables on this column.

List the first 5 candidate names (`cand_name`) in reverse lexicographic order by `cand_name`, along with their corresponding committee names. Only select rows that have a matching `cand_id` in both tables.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORN, ROBERT ERWIN</td>
      <td>CONSTITUTIONAL COMMITTEE</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2b
-->

In [50]:
query_q2b='''
SELECT ..., ...
FROM cand c1 ... JOIN comm c2 ON ...
...
...
'''

# BEGIN SOLUTION NO PROMPT
# Solution 1
query_q2b='''
SELECT c1.cand_name, c2.cmte_nm
FROM cand c1 INNER JOIN comm c2 ON c1.cand_id = c2.cand_id
ORDER BY c1.cand_name DESC
LIMIT 5
'''

# Solution 2
query_q2b='''
SELECT cand.cand_name, comm.cmte_nm
FROM cand INNER JOIN comm ON cand.cand_id = comm.cand_id
ORDER BY cand.cand_name DESC
LIMIT 5
'''
# END SOLUTION

res_q2b = pd.read_sql(query_q2b, engine)
res_q2b

,cand_name,cmte_nm
0,"ZUTLER, DANIEL PAUL MR",CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT
1,"ZUMWALT, JAMES",ZUMWALT FOR CONGRESS
2,"ZUKOWSKI, ANDREW GEORGE",ZUKOWSKI FOR CONGRESS
3,"ZUCCOLO, JOE",JOE ZUCCOLO FOR CONGRESS
4,"ZORN, ROBERT ERWIN",CONSTITUTIONAL COMMITTEE


In [ ]:
grader.check("q2b")

### Question 2c

As in the previous part, list the first 5 candidate names (`cand_name`) by reverse lexicographic order and their corresponding committee names (`cmte_nm`). This time, include all candidate names. If there is no matching `cand_id` in the `comm` table, then `cmte_nm` should be NULL.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORNOW, TODD MR</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2c
-->

In [51]:
query_q2c='''
SELECT ..., ...
FROM cand c1 ... JOIN comm c2 ON ...
...
...
'''

# BEGIN SOLUTION NO PROMPT
query_q2c='''
SELECT c1.cand_name, c2.cmte_nm
FROM cand c1 LEFT OUTER JOIN comm c2 ON c1.cand_id = c2.cand_id
ORDER BY c1.cand_name DESC
LIMIT 5
'''
# just change "INNER JOIN" to "LEFT OUTER JOIN"
# END SOLUTION

res_q2c = pd.read_sql(query_q2c, engine)
res_q2c

,cand_name,cmte_nm
0,"ZUTLER, DANIEL PAUL MR",CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT
1,"ZUMWALT, JAMES",ZUMWALT FOR CONGRESS
2,"ZUKOWSKI, ANDREW GEORGE",ZUKOWSKI FOR CONGRESS
3,"ZUCCOLO, JOE",JOE ZUCCOLO FOR CONGRESS
4,"ZORNOW, TODD MR",None


In [ ]:
grader.check("q2c")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)